In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import csv 
%matplotlib inline

In [ ]:
%lsmagic

In [ ]:
def get_data(symbol, exchange='bitfinex', after='2020-09-01'):
    url = 'https://api.cryptowat.ch/markets/{exchange}/{symbol}usd/ohlc'.format(
        symbol=symbol, exchange=exchange)
    try:
        resp = requests.get(url, params={
            'periods': '3600',
            'after': str(int(pd.Timestamp(after).timestamp()))
        })
    except:
        print('Something wrong with resp')
    resp.raise_for_status()
    data = resp.json()
    print(data)
    df = pd.DataFrame(data['result']['3600'], columns=[
        'CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'NA'
    ])
    df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    df.set_index('CloseTime', inplace=True)
    return df
    

In [ ]:
last_week = (pd.Timestamp.now() - pd.offsets.Day(7))
last_week

In [ ]:
btc = get_data('btc', 'bitstamp', after=last_week)

In [ ]:
btc

In [ ]:
btc['ClosePrice'].plot(figsize=(15, 7))

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
p1 = figure(x_axis_type="datetime", title="Crypto Prices", width=800)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Price'

p1.line(btc.index, btc['ClosePrice'], color='#f2a900', legend='Bitcoin')
#p1.line(eth.index, eth['ClosePrice'], color='#A6CEE3', legend='Ether')

p1.legend.location = "top_left"

show(p1)

In [ ]:
writer = pd.ExcelWriter('cryptos.xlsx')

In [ ]:
btc.to_excel(writer, sheet_name='Bitcoin')

In [ ]:
writer.save()

In [61]:
def get_assets():
    url = 'https://api.cryptowat.ch/markets'
    response = requests.get(url)
    response.raise_for_status()
    print(type(response))
    data = response.json()
    exmaple = data['result']
    x = 0
    with open('pairs_crypto.csv', 'w', newline='') as csvfile:
        header = ['exchange', 'pairs', 'route','price', 'high', 'low', 'percentage', 'absolute']
        writer = csv.writer(csvfile, delimiter=',', quotechar='|')
        writer.writerow(header)
        for pairs in exmaple:
            x += 1
            if x == 500:
                break
            if pairs['active'] == True:
                print(pairs)
                url_summery = 'https://api.cryptowat.ch/markets/{exc}/{pai}/summary'.format(exc=pairs['exchange'], pai=pairs['pair'])
                resp_summ = requests.get(url_summery)
                resp_summ.raise_for_status()
                resp_summ = resp_summ.json()
                resp_summ = resp_summ['result']['price']
#                 print(resp_summ)
                writer.writerow([pairs['exchange'], pairs['pair'], pairs['route'], round(resp_summ['last'],2),
                                 round(resp_summ['high'],2), round(resp_summ['low'],2), round(resp_summ['change']['percentage'],2),
                                 round(resp_summ['change']['absolute'],2)])
#     route = exmaple['route']
#     resp = requests.get(route)
#     resp.raise_for_status()
#     new_data = resp.json()
#     print(new_data)
#     print(type(data))
#     print(data)
#     print(exmaple)

In [62]:
get_assets()

<class 'requests.models.Response'>
{'id': 1, 'exchange': 'bitfinex', 'pair': 'btcusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/btcusd'}
{'id': 2, 'exchange': 'bitfinex', 'pair': 'ltcusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/ltcusd'}
{'id': 3, 'exchange': 'bitfinex', 'pair': 'ltcbtc', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/ltcbtc'}
{'id': 4, 'exchange': 'bitfinex', 'pair': 'ethusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/ethusd'}
{'id': 5, 'exchange': 'bitfinex', 'pair': 'ethbtc', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/ethbtc'}
{'id': 6, 'exchange': 'bitfinex', 'pair': 'etcbtc', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/etcbtc'}
{'id': 7, 'exchange': 'bitfinex', 'pair': 'etcusd', 'active': True, 'route': 'https://api.cryptowat.ch/markets/bitfinex/etcusd'}
{'id': 14, 'exchange': 'bitfinex', 'pair': 'eosbtc', 'active':